In [1]:
'''
Run GNN policy on robot
This one has the control inputs stripped to frame with (x y yaw) removed
Uses the trained policy from the velocity input



'''

# load libraries
import torch
from robot_env import robot_env
import numpy as np
import pgnn_control2 as pgnnc
from utils import to_tensors, combine_state, wrap_to_pi, rotate, create_control_inputs2

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# init env
# use the env to get the size of the inputs and outputs
# env = robot_env(show_GUI = False)
env = robot_env(show_GUI = True)
env.reset_terrain()

    
USE_JOY = True
if USE_JOY:
    from vrjoystick import init_joystick, read
    joy = init_joystick()
    


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Logitech Logitech Dual Action


In [2]:
# urdf_name = 'llllll'
# urdf_name = 'llwwll'
# urdf_name = 'lnllnl'
# urdf_name = 'lnwwnl'
# urdf_name = 'wnwwnw'

# urdf_names = ['llllll', 'llwwll', 
# 'lwllwl', 'lwwwwl', 
# 'lnllnl', 'lnwwnl',
# 'wlwwlw',  'wwllww', 
# 'wwwwww',  'wnllnw', 
#  'wllllw', 'wnwwnw']


# urdf_names = ['llllll', 'wnwwnw']

# urdf_name = urdf_names[-1]
# urdf_name = urdf_names[1]
# urdf_name = urdf_names[5]
# urdf_name = 'llwwll'

# urdf_name = 'lllwnw' #'llwwnw'

# os.path.isfile(urdf_name)

folder = 'mbrl_v7_test1/'
folder = 'mbrl_v8_test9/'
# folder = ''
# PATH = urdf_name + '_control_gnn_vel.pt'
# PATH =  'multidesign_control_gnn250_v3.pt'
PATH =  'multidesign_control_iter3.pt'
# PATH =  'multidesign_control_iter3_500hidden.pt'
# PATH =  'multidesign_control_v0_gnn4.pt'
# PATH = 'trained_controllers/'+PATH
PATH = folder + PATH

save_dict = torch.load( PATH, map_location=lambda storage, loc: storage)
urdf_names = save_dict['urdf_names']
print(urdf_names)
urdf_name = 'wnwwnw'
# urdf_name = 'wwwwww'
# urdf_name = 'llllll'
# urdf_name = 'wnllnw'
# urdf_name = 'lnllnl'
# urdf_name = 'lwwwnl'

# urdf_names_out = ['nwnnwn', 'nwllwn', 'lnwnln', 'wnwnwn'] 

gnn_state_dict= save_dict['gnn_state_dict'] 
internal_state_len = save_dict['internal_state_len'] 
message_len = save_dict['message_len'] 
hidden_layer_size = save_dict['hidden_layer_size']
goal_len =3

print(save_dict['comment'])

env.reset_robot(urdf_name=urdf_name, randomize_start=False)

attachments = env.attachments
modules_types = env.modules_types
print('attachments: ' + str(attachments))
print('modules_types: ' + str(modules_types))
n_modules = len(modules_types)

env_state_init = env.get_state()
module_state_len = []
for s in env_state_init:
    module_state_len.append(len(s))

state_len= np.sum(module_state_len)
action_len = env.num_joints
module_action_len = list(np.diff(env.action_indexes))

module_sa_len = module_state_len+ module_action_len

gnn_nodes = pgnnc.create_GNN_nodes(internal_state_len, 
                                   message_len, hidden_layer_size,
                                   device, goal_len=goal_len, body_input = True)
pgnnc.load_state_dicts(gnn_nodes, save_dict['gnn_state_dict']) 

# create module containers for the nodes
modules = []
for i in range(n_modules):
    modules.append(pgnnc.Module(i, gnn_nodes[modules_types[i]], device))

['llllll', 'lnwwnl', 'llwwll', 'lnllnl', 'lwllwl', 'lwwwwl', 'wlwwlw', 'wwllww', 'wwwwww', 'wnllnw', 'wllllw', 'wnwwnw']

attachments: [[1, None, 2, 3, None, 4], [0], [0], [0], [0]]
modules_types: [0, 2, 2, 2, 2]


In [4]:
env.p.resetDebugVisualizerCamera(2.1,0,-65,[0,0,0.2],physicsClientId=env.physicsClient) 
# env.p.resetDebugVisualizerCamera(2.1,0,-89.999,[0,0,0.2],physicsClientId=env.physicsClient) 
env.reset_robot(urdf_name=urdf_name, randomize_start=False, start_xyyaw=[0,0,0])
# vid_fname=urdf_name+'_steer_vel.mp4'
# env.start_video_log(fileName=vid_fname)
# for step in range(150):
robot_button_mapping = [0,1,2,3,4,10,11]
n_buttons_to_check = len(robot_button_mapping)
vxy_scale = 20*(20/240)*0.314/0.75
vyaw_scale = 20*(20/240)*1.1/0.75

env_state = env.get_state()
last_states = [smm.to(device) for smm in to_tensors(env_state)]
# last_action = np.zeros(action_len)
tau_list = []

buttons = np.zeros(12)
for step in range(10000):
    
    if USE_JOY:
        axes, buttons, povs = read(joy)
        axes = np.array(axes)
        axes[np.abs(axes)<0.01] = 0
        desired_xyyaw = np.array([-axes[1]*vxy_scale,
                                  -axes[0]*vxy_scale,
                                  -axes[2]*vyaw_scale])
        if buttons[9]==1 or buttons[8]==1:
            break
        
        if np.any(buttons[0:n_buttons_to_check]) or buttons[10] or buttons[11]:
            if np.any(buttons[0:n_buttons_to_check]):
                pressed = np.where(buttons[0:n_buttons_to_check])[0][0]
                urdf_name = urdf_names[robot_button_mapping[pressed]]
            else:
                ind = urdf_names.index(urdf_name)
                urdf_name = urdf_names[(ind+1) % len(urdf_names)]
#             urdf_name = urdf_names_out[pressed]
            env.reset_terrain()
            env.reset_robot(urdf_name=urdf_name, randomize_start=False, start_xyyaw=[0,0,0])
            attachments = env.attachments
            modules_types = env.modules_types
            n_modules = len(modules_types)
            env_state_init = env.get_state()
            module_state_len = []
            for s in env_state_init:
                module_state_len.append(len(s))
            state_len= np.sum(module_state_len)
            action_len = env.num_joints
            module_action_len = list(np.diff(env.action_indexes))
            module_sa_len = module_state_len+ module_action_len
            # create module containers for the nodes
            modules = []
            for i in range(n_modules):
                modules.append(pgnnc.Module(i, gnn_nodes[modules_types[i]], device))
            tau_list = []
            last_states = [smm.to(device) for smm in to_tensors(env_state_init)]


    else:
        desired_xyyaw =  np.array([1,0, 0])

        
    chassis_yaw = env.pos_rpy[-1]
    vect1 = np.array([desired_xyyaw[0]*np.cos(chassis_yaw) - desired_xyyaw[1]*np.sin(chassis_yaw),
            desired_xyyaw[0]*np.sin(chassis_yaw) + desired_xyyaw[1]*np.cos(chassis_yaw),
            0] )
    vect2 = np.array([np.cos(desired_xyyaw[2]+chassis_yaw),
                      np.sin(desired_xyyaw[2]+chassis_yaw), 
                      0])*np.abs(desired_xyyaw[2])
    env.draw_body_arrows([vect1*0.5/vxy_scale, 
                          0.5*vect2/vyaw_scale],
                         [[0,0,0], [0,0,1]])

    
    env_state = env.get_state()
    states = [smm.to(device) for smm in to_tensors(env_state)]
    
#     test_state = env_state
#     test_state[2] = 1
#     env.set_state(test_state)

    
    
    

    # heading and yaw here are wrt body frame
    goals = torch.tensor(desired_xyyaw, dtype=torch.float32, device=device).unsqueeze(0)

    node_inputs = create_control_inputs(last_states, states, goals, rotate_goals = False)
    

    for module in modules: # this prevents the LSTM in the GNN nodes from 
        # learning relations over time, only over internal prop steps.
        module.reset_hidden_states(1) 

    with torch.no_grad():
        out_mean, out_var = pgnnc.run_propagations(
            modules, attachments, 2, node_inputs, device)
        u_out_mean = []
        tau_out_mean = []
        for mm in range(n_modules):
            u_out_mean.append(out_mean[mm][:,:module_action_len[mm]])
            tau_out_mean.append(out_mean[mm][:,module_action_len[mm]:])
        action = torch.cat(u_out_mean,-1).squeeze().numpy()
        tau =  torch.cat(tau_out_mean,-1).squeeze().cpu().numpy()
        tau_list.append(tau)
#         print(np.array2string(tau, precision=2))
    
    env.step(action)
    

    
        
    if np.dot([0,0,1], env.z_axis)<0:
        env_yaw = env.pos_rpy[-1]
        env.reset_robot(urdf_name=urdf_name, randomize_start=True)
        env_state = env.get_state()
        states = [smm.to(device) for smm in to_tensors(env_state)]
        
    if np.sqrt(env.pos_xyz[0]**2+ env.pos_xyz[1]**2)>2:
        env_state = env.get_state()
        env_state[0][0:2] = 0 
        env.set_state(env_state)
    
    last_states = [s.clone() for s in states]

# env.stop_video_log()

TypeError: create_control_inputs() got multiple values for argument 'rotate_goals'

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

tau_array = np.stack(tau_list)
all_joint_inds = list(range(env.num_joints))
jnames = []
for i in all_joint_inds[1::3]:
    jnames.append(str(i))
    plt.plot(tau_array[:,i])
plt.legend(jnames)



# convert video to slow down to correct speed with ffmpeg
# import subprocess
# cmd_status = subprocess.call(['ffmpeg', '-i',
#                   vid_fname, '-filter:v',
#                   'setpts=2.0*PTS',
#                   '2x_' + vid_fname, '-y'])

In [ ]:
# env.p.getDebugVisualizerCamera()
# width, height, viewmatrix, projmatrix, cameraup, cameraforward, 
# horizontalvec, verticalvec, yaw, pitch,dist, target

In [ ]:
10*np.exp(-np.array([0, 1000, 5000])/2000)